In [1]:
%run download_geographic_refs.ipynb
%run set_up.py 

import geopandas as gpd
import numpy as np
import pandas as pd #require pip install pyarrow
from shapely.geometry import Point
import re
import warnings

verbose=True

In [7]:
test_pts_path = os.path.join(datdir, 'test_gages', 'test_gages.shp')

wbd_path = os.path.join(nhd_dir, 'WBD_National_GDB.gdb')
hu12_parquet = os.path.join(nhd_dir, 'wbd_hu12list.parquet')
basinatlas_path = os.path.join(hydroatlas_dir,  'BasinATLAS_v10.gdb')
basinatlas11_parquet = os.path.join(hydroatlas_dir, 'basinatlas_lev11_idlist.parquet')
#geoglows_vpu_path = os.path.join(geoglows_dir, 'vpu-boundaries.gpkg')
gadm_path = os.path.join(gadm_dir, 'gadm_410-levels.gpkg')

In [8]:
def _expand_basin_idlist(in_id_list,
                         in_refids_parquet,
                         refids_col,
                         out_id_range):
    """
    Expands a list of basin IDs by extracting IDs at different levels from a reference Parquet file.

    Args:
        in_id_list (list): List of input basin IDs.  Can be integers or strings.
        in_refids_parquet (str): Path to the reference Parquet file.
        refids_col (str): Name of the column in the Parquet file containing the full basin IDs.
        out_id_range (list): List of integer levels to extract.  e.g., [6, 9, 12]

    Returns:
        pandas.DataFrame: DataFrame containing the expanded basin IDs.  The returned DataFrame
                          will have columns named based on `refids_col` and levels in `out_id_range`.
                          The data type of the output columns will match the input type of `in_id_list`.

    Raises:
        TypeError: If the length of input IDs is inconsistent with the length in reference table.
        FileNotFoundError: If the input Parquet file does not exist.
        ValueError:  If `out_id_range` contains values greater than the maximum ID length.
                   Or if an empty DataFrame is returned by filtering.
    """

    id_all_pd = pd.read_parquet(in_refids_parquet)

    # Determine input type and maximum ID length
    in_id_type = type(in_id_list[0])
    if in_id_type == str:
      in_id_len = len(in_id_list[0])
    elif in_id_type == int:
      in_id_len = len(str(in_id_list[0]))
    else:
      raise TypeError("in_id_list must be a list of strings or integers")

    # Determine the reference ID type and length
    first_ref_id = id_all_pd[refids_col].iloc[0] #Get first item in col
    refid_type = type(first_ref_id)

    if refid_type == str:
      refid_len = len(first_ref_id)
    elif pd.api.types.is_integer_dtype(refid_type) or  refid_type == np.int64:
      refid_len = len(str(first_ref_id))
    else:
      raise TypeError(f"{refids_col} must contain strings or integers")


    if in_id_len > refid_len:
        raise ValueError(f"Input ID length ({in_id_len}) is greater than reference ID length ({refid_len})")

    # --- Input Validation on out_id_range ---
    if any(level > refid_len for level in out_id_range):
        raise ValueError(f"out_id_range values cannot exceed reference ID length ({refid_len})")

    # --- Filtering ---
    # Convert to string for consistent prefix matching, then convert back later
    id_all_pd[refids_col] = id_all_pd[refids_col].astype(str)
    in_id_list_str = [str(x) for x in in_id_list]

    id_pd = id_all_pd[id_all_pd[refids_col].str.startswith(tuple(in_id_list_str))].copy()

    if id_pd.empty:
        raise ValueError("No matching IDs found. Check in_id_list and refids_col.")

    # --- Column Expansion ---
    colroot = re.sub(r'[0-9]+', '', refids_col)
    for id_level in out_id_range:
        col_name = f'{colroot}{id_level}'
        if col_name not in id_pd.columns:
            id_pd.loc[:, col_name] = id_pd[refids_col].str[:id_level]
            # Convert back to original type, if necessary
            if in_id_type == int:
                id_pd.loc[:, col_name] = pd.to_numeric(id_pd[col_name],
                                                       errors='raise') 

    # Convert refids_col back to original type
    if pd.api.types.is_integer_dtype(refid_type) or  refid_type == np.int64:
        id_pd[refids_col] = pd.to_numeric(id_pd[refids_col], 
                                          errors='raise')

    return id_pd

In [9]:
def _format_gdf_tojoin(in_xytab=None, lon_col=None, lat_col=None, 
                       in_vector=None, hull=True):
    #Read xy table
    if in_xytab:
        points_df = pd.read_table(point_locations_path)
        gdf_to_join = gpd.GeoDataFrame(
            points_df,
            geometry=[Point(xy) for xy 
                      in zip(points_df[lon_col], points_df[lat_col])]
        )
    
    #Read vector layer
    if in_vector:
        gdf_to_join = gpd.read_file(in_vector)

    if hull:
        # Create convex hull using union_all() (current recommended method)
        gdf_to_join = gpd.GeoDataFrame(
            geometry=[gdf_to_join.geometry.union_all().convex_hull], 
            crs=gdf_to_join.crs
        )
    return(gdf_to_join)

In [14]:
#Create a list of PFAF_ID for basins level 11
def create_basinatlas11_list(basinatlas_path, 
                             out_basinatlas11_parquet, 
                             verbose=True):
    if not os.path.exists(out_basinatlas11_parquet):
        if verbose:
            print(f'Generating a list of PFAF ID level 11 and saving it to \
            {out_basinatlas11_parquet}')
            
        basinatlas11_list = gpd.read_file(
            filename=basinatlas_path, 
            layer='BasinATLAS_v10_lev11', 
            columns=['PFAF_ID'],
            rows=1031785,
            ignore_geometry=True).\
        astype(pd.Int64Dtype()).\
        rename(columns={"PFAF_ID": "PFAF_ID11"})
        
        basinatlas11_list.to_parquet(out_basinatlas11_parquet)
    else:
        basinatlas11_list = pd.read_parquet(out_basinatlas11_parquet)
    return(basinatlas11_list)

In [ ]:
def get_matching_hydrobasin(in_basinatlas_path,
                            in_xytab=None, lon_col=None, lat_col=None,
                            in_vector=None, 
                            in_id_list=None, 
                            in_refids_parquet=None,
                            hull=True, sjoin_predicate='intersects'):
    #If points or polygons are provided ----------------------------------------
    if in_xytab or in_vector:
        gdf_to_join = _format_gdf_tojoin(in_xytab, lon_col, lat_col, 
                                         in_vector, 
                                         hull)
        
        #Reach NHD WBD
        bas_lev6 = gpd.read_file(filename=in_basinatlas_path, 
                                 layer='BasinATLAS_v10_lev06',
                                 columns=['PFAF_ID']
                                ).rename(columns={"PFAF_ID": "PFAF_ID6"})
    
        #Spatially join to hydrologic units
        matched_bas = gpd.sjoin(gdf_to_join.to_crs(crs=bas_lev6.crs), 
                               bas_lev6, 
                               how='left', 
                               predicate=sjoin_predicate)
        in_id_list = matched_bas.PFAF_ID6.tolist()

    pfaf_pd = _expand_basin_idlist(
        in_id_list, 
        in_refids_parquet, 
        refids_col='PFAF_ID11', 
        out_id_range=range(3, 12))

    return(pfaf_pd)

In [ ]:
def get_hydroatlas_data():
    print('Getting HydroATLAS data')

In [12]:
#Create a list of NHD HUC12
def create_huc12_list(wbd_path, 
                      out_hu12_parquet,
                      verbose=True):
    if not os.path.exists(out_hu12_parquet):
        if verbose:
            print(f'Generating a list of HUC 12 and saving it to \
            {out_hu12_parquet}')
        wbdhu12_list = gpd.read_file(filename=wbd_path, 
                                     layer='WBDHU12', 
                                     rows=105000,
                                     columns=['huc12'],
                                     ignore_geometry=True)
        wbdhu12_list.to_parquet(out_hu12_parquet)
    else:
        wbdhu12_list = pd.read_parquet(out_hu12_parquet)
    return(wbdhu12_list)

In [ ]:
def get_matching_NHD_HU(in_wbd_path,
                        in_xytab=None, lon_col=None, lat_col=None,
                        in_vector=None, 
                        in_id_list=None, 
                        in_refids_parquet=None,
                        hull=True, sjoin_predicate='intersects'):

    #If points or polygons are provided ----------------------------------------
    if in_xytab or in_vector:
        gdf_to_join = _format_gdf_tojoin(in_xytab, lon_col, lat_col, 
                                         in_vector, 
                                         hull)
        
        #Get NHD WBD
        wbdhu6 = gpd.read_file(filename=in_wbd_path, 
                               layer='WBDHU6',
                               columns=['huc6']
                              )
    
        #Spatially join to hydrologic units
        matched_nhd = gpd.sjoin(gdf_to_join.to_crs(crs=wbdhu6.crs), 
                               wbdhu6, 
                               how='left', 
                               predicate=sjoin_predicate)
        in_id_list = matched_nhd.huc6.values.tolist()
        
    huc_pd = _expand_basin_idlist(
        in_id_list, 
        in_refids_parquet, 
        refids_col='huc12', 
        out_id_range=range(2, 14, 2))

    return(huc_pd)

In [ ]:
def get_nhd_hydronyms(in_hucs, in_wbd_path,  out_dir,
                      huc_range=range(2, 14, 2),
                      flatten=True,
                      verbose=True
                    ):
    print('Getting NHD basin names')

    #If panda dataframe
    #Get basin names------------------------------------------------------------
    for coln in in_hucs.columns:
        huc_len = re.sub(r'[a-zA-Z]+', '', coln)
        if int(huc_len) is None:
            raise ValueError(f"HUC level cannot be extracted from {coln}")
        if int(huc_len) in huc_range:
            wbd = gpd.read_file(filename=in_wbd_path, 
                                layer=f'WBDHU{huc_len}',
                                columns=[coln, 'name'],
                                ignore_geometry=True
                               )
            in_hucs = in_hucs.merge(wbd, on=coln, how='left').\
            rename(columns={"name": f"{coln}_name"})

    #Download data by HU4 if needed
    huc4_list = in_hucs.huc4.unique()
    nhd_huc4_pathdict = {}
    for huc in huc4_list:
        download_nhdplus_hr_hu4(
            hu4=huc, 
            out_dir=out_dir, 
            verbose=False
        )
        nhd_huc4_pathdict[huc] = os.path.join(
            out_dir,
            f'NHDPLUS_H_{huc}_HU4_GDB.gdb')
    #print(nhd_huc4_pathdict)
    
    #Get river names------------------------------------------------------------
    #NHD flow line types: FCode attribute to subset
    # 46000: Stream/River
    # 46003: Stream/River: Hydrographic Category = Intermittent
    # 46006: Stream/River: Hydrographic Category = Perennial
    # 46007: Stream/River: Hydrographic Category = Ephemeral
    # 55800: Artificial path'''
    fcode_sel_list = [46000, 46006, 46003, 46007, 55800]
    huc4_rivnames_dict = {}

    for huc4 in nhd_huc4_pathdict:
        if verbose:
            print(huc4)
        #Read flowlines
        #NHD uses geometries with measured (M) values but not supported by pyogrio
        #underlying geopandas by default (faster than fiona)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning, module="pyogrio.raw")
            
            flowlines_gpd = gpd.read_file(
                filename=nhd_huc4_pathdict[huc4], 
                layer='NHDFlowline',
                columns=['NHDPlusID', 'ReachCode', 'GNIS_Name', 'FCode'],
                ignore_geometry=True
            )
            
        vaa_pd = gpd.read_file(
            filename=nhd_huc4_pathdict[huc4], 
            layer='NHDPlusFlowlineVAA',
            columns=['NHDPlusID', 'StreamOrde'],
            ignore_geometry=True
        )


        #reachcode: The first eight digits are the WBD_HUC8.
        #The next six digits are randomly assigned, 
        #sequential numbers that are unique within a HUC8.
        flowlines_gpd['huc8'] = flowlines_gpd['ReachCode'].str[:8] 
        huc8_sel = in_hucs[in_hucs['huc4']==huc4]['huc8'].unique()
        flowlines_sub = flowlines_gpd[flowlines_gpd['huc8'].isin(huc8_sel)].\
        merge(vaa_pd, how='inner', on='NHDPlusID')
    
        rivnames = flowlines_sub[(
            (flowlines_sub['FCode'].isin(fcode_sel_list)) 
            & (flowlines_sub['StreamOrde'] >=6)
            & (flowlines_sub['GNIS_Name'].notna())
        )].GNIS_Name.unique()

        huc4_rivnames_dict[huc4] = rivnames



    if flatten:
        #Create a set with all unique hydronyms from basins and rivers from NHD
        test_nhd_hydronyms_set = set([
            *pd.melt(in_hucs, 
                     value_vars=[f'huc{lev}_name' for lev in huc_range]
                    ).value.unique(),
            *set({x for v in huc4_rivnames_dict.values() for x in v})
        ])
        return(test_nhd_hydronyms_set)
    else:
        #Return dictionary with basin names and river names
        out_dict = {}
        out_dict['basins_all_pd'] = in_hucs
        out_dict['rivers_huc4_dict'] = huc4_rivnames_dict
        return(out_dict)

In [ ]:
def get_nhd_data(in_hucs_pd, out_dir, verbose=True):
    print('Getting NHD data for all HU4')
    
    #Download data by HU4 if needed
    huc4_list = in_hucs_pd.huc4.unique()
    nhd_huc4_pathdict = {}
    for huc in huc4_list:
        download_nhdplus_hr_hu4(
            hu4=huc, 
            out_dir=out_dir, 
            verbose=False
        )
        nhd_huc4_pathdict[huc] = os.path.join(
            out_dir,
            f'NHDPLUS_H_{huc}_HU4_GDB.gdb')

    nhdplus_huc4_dict = {}
    for huc4 in nhd_huc4_pathdict:
        if verbose:
            print(huc4)
            
        #Read flowlines
        #NHD uses geometries with measured (M) values but not supported by pyogrio
        #underlying geopandas by default (faster than fiona)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning, module="pyogrio.raw")
            
            flowlines_gpd = gpd.read_file(
                filename=nhd_huc4_pathdict[huc4], 
                layer='NHDFlowline',
                ignore_geometry=False
            )
        vaa_pd = gpd.read_file(
            filename=nhd_huc4_pathdict[huc4], 
            layer='NHDPlusFlowlineVAA',
            ignore_geometry=False
        )

        flowlines_vaa = flowlines_gpd.merge(
            vaa_pd, how='inner', on='NHDPlusID', suffixes=('', '_vaa'))

        nhdplus_huc4_dict[huc4] = flowlines_vaa
    
    return(nhdplus_huc4_dict)

In [ ]:
def get_geoglows_vpu(in_geoglows_vpu_path,
                     in_xytab=None, lon_col=None, lat_col=None,
                     in_vector=None, 
                     in_id_list=None, 
                     in_refids_parquet=None,
                     hull=True, sjoin_predicate='intersects'):
    
    if in_xytab or in_vector:
        gdf_to_join = _format_gdf_tojoin(in_xytab, lon_col, lat_col, 
                                         in_vector, 
                                         hull)
        
        #Reach NHD WBD
        vpus = gpd.read_file(filename=in_geoglows_vpu_path, 
                                 layer='vpu-boundaries',
                                 columns=['VPU']
                                )
    
        #Spatially join to hydrologic units
        matched_vpus = gpd.sjoin(gdf_to_join.to_crs(crs=vpus.crs), 
                                 vpus, 
                                 how='left', 
                                 predicate=sjoin_predicate)
        vpu_list = matched_vpus.VPU.tolist()

    return(vpu_list)

In [32]:
def get_gadm_lev1_dict(in_gadm_path,
                  in_xytab=None, lon_col=None, lat_col=None,
                  in_vector=None, 
                  in_id_list=None, 
                  in_refids_parquet=None,
                  hull=True, sjoin_predicate='intersects'):
    if in_xytab or in_vector:
        gdf_to_join = _format_gdf_tojoin(in_xytab, lon_col, lat_col, 
                                         in_vector, 
                                         hull)
        
        #Get GADM data
        gadm_gpd = gpd.read_file(filename=in_gadm_path, layer='ADM_1')
    
        #Spatially join to hydrologic units
        matched_adm_units = gpd.sjoin(
            gadm_gpd,
            gdf_to_join.to_crs(crs=gadm_gpd.crs), 
            how='inner', 
            predicate=sjoin_predicate)
        #print(matched_adm_units)

    return(matched_adm_units)

In [ ]:
# #Run functions
# hu12_list = create_huc12_list(wbd_path, hu12_parquet)

# basinatlas11_list = create_basinatlas11_list(
#     basinatlas_path, 
#     basinatlas11_parquet)

# test_huc_pd = get_matching_NHD_HU(
#     in_wbd_path=wbd_path,
#     in_vector=test_pts_path,
#     in_refids_parquet=hu12_parquet,
#     hull=True,
#     sjoin_predicate='intersects'
# )
# #print(test_huc_pd)
# #in_id_list = in_umrb_huc4s = [f'07{str(i).zfill(2)}' for i in range(2,15)]

# test_nhd_hydronyms = get_nhd_hydronyms(
#     in_hucs=test_huc_pd,
#     in_wbd_path=wbd_path,
#     out_dir = os.path.join(nhd_dir, 'nhdplus_hr'),
#     huc_range=[2, 4, 6],
#     verbose=False
# )
# len(test_nhd_hydronyms)

# test_pfaf_pd = get_matching_hydrobasin(
#     in_basinatlas_path=basinatlas_path,
#     in_vector=test_pts_path,
#     #in_id_list=None, 
#     in_refids_parquet=basinatlas11_parquet,
#     hull=True,
#     sjoin_predicate='intersects'
# )
# #print(test_pfaf_pd)

# test_pfaf_pd_idlist = get_matching_hydrobasin(
#     in_basinatlas_path=basinatlas_path,
#     in_id_list=[742873, 742875, 742876], 
#     in_refids_parquet=basinatlas11_parquet
# )
# #print(test_pfaf_pd_idlist)


# test_vpu_list = get_geoglows_vpu(
#     in_geoglows_vpu_path=geoglows_vpu_path,
#     in_vector=test_pts_path,
#     hull=True,
#     sjoin_predicate='intersects'
# )
# print(test_vpu_list)

# test_gadm_lev1 = get_gadm_lev1_dict(
#     in_gadm_path=gadm_path,
#     in_vector=test_pts_path,
#     hull=True,
#     sjoin_predicate='intersects'
# )

In [ ]:
################IN DEVELOPMENT ###############################

In [ ]:
def get_geoglows_hydronyms(in_geoglows_path, verbose=True):

    country_tab_path = os.path.join(in_geoglows_path, 
                                    'tables', 'v2-countries-table.parquet')
    country_pd = pd.read_parquet(country_tab_path)

    meta_tab_path = os.path.join(in_geoglows_path, 
                                    'tables', 'package-metadata-table.parquet')
    meta_pd = pd.read_parquet(meta_tab_path)

    model_tab_path = os.path.join(in_geoglows_path, 
                                    'tables', 'v2-model-table.parquet')
    model_pd = pd.read_parquet(model_tab_path)

    print('Getting geoglows river names')
    for vpu in test_vpu_list[0]:
        streams_gpd= gpd.read_file(
                filename=nhd_huc4_pathdict[huc4], 
                layer='NHDFlowline',
                ignore_geometry=False
        )
          
# geoglows_path = os.path.join(datdir, 'geoglows')
# get_geoglows_hydronyms(in_geoglows_path=geoglows_path, verbose=True)


